In [1]:
#https://qiita.com/shnchr/items/bdd4810b3477aa977413
import warnings
warnings.filterwarnings("ignore")
from pycaret.classification import *
import pandas as pd

データを読み込む

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
print("Train:",train.shape, "Test:",test.shape)

Train: (27128, 18) Test: (18083, 17)


データを見る

In [9]:
#import pandas_profiling
#train.profile_report()

前処理（欠損値処理、データ分割など）

In [4]:
xp1=setup(train, target='y')

 
Setup Succesfully Completed!


,Description,Value
0,session_id,6550
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(27128, 18)"
4,Missing Values,False
5,Numeric Features,8
6,Categorical Features,9
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


モデルを比較

In [5]:
compare_models(sort='AUC')

IntProgress(value=0, description='Processing: ', max=170)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:50:51
Status,. . . . . . . . . . . . . . . . . .,Compiling Final Model
ETC,. . . . . . . . . . . . . . . . . .,0.0 Seconds Remaining


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Light Gradient Boosting Machine,0.9036,0.9293,0.4604,0.6186,0.5276,0.4752
1,Gradient Boosting Classifier,0.9049,0.9220,0.3974,0.6553,0.4942,0.4452
2,Extreme Gradient Boosting,0.9036,0.9215,0.3650,0.6590,0.4690,0.4210
3,Ada Boost Classifier,0.8990,0.9072,0.3780,0.6115,0.4669,0.4146
4,Linear Discriminant Analysis,0.8986,0.9046,0.4249,0.5939,0.4950,0.4404
5,Extra Trees Classifier,0.8958,0.9000,0.3042,0.6107,0.4051,0.3551
6,Random Forest Classifier,0.8981,0.8876,0.3164,0.6298,0.4204,0.3712
7,Logistic Regression,0.8899,0.8477,0.2070,0.5813,0.3044,0.2593
8,Quadratic Discriminant Analysis,0.8715,0.8366,0.4798,0.4543,0.4663,0.3934
9,Naive Bayes,0.8581,0.8194,0.5126,0.4146,0.4582,0.3777


AttributeError: module 'pandas' has no attribute 'SparseDtype'

パラメータチューニング

In [6]:
tuned_model = tune_model(estimator='lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.9063,0.9175,0.4505,0.6410,0.5291,0.4788
1,0.9184,0.9235,0.5225,0.7030,0.5995,0.5551
2,0.9026,0.9239,0.4189,0.6242,0.5013,0.4497
3,0.9042,0.9342,0.5180,0.6053,0.5583,0.5049
4,0.9026,0.9183,0.4685,0.6082,0.5293,0.4759
5,0.9047,0.9371,0.4459,0.6306,0.5224,0.4712
6,0.9021,0.9154,0.4369,0.6139,0.5105,0.4578
7,0.8957,0.9220,0.3857,0.5850,0.4649,0.4098
8,0.9068,0.9262,0.4664,0.6420,0.5403,0.4898
9,0.9046,0.9249,0.4234,0.6395,0.5095,0.4591


結果を可視化

In [20]:
evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

テストデータで予測

In [22]:
final_lr = finalize_model(tuned_model)
pred = predict_model(final_lr, data=test)
pred.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Label,Score
0,1,30,management,single,tertiary,no,1028,no,no,cellular,4,feb,1294,2,-1,0,unknown,1,0.5441
1,2,39,self-employed,single,tertiary,no,426,no,no,unknown,18,jun,1029,1,-1,0,unknown,1,0.6507
2,3,38,technician,single,tertiary,no,-572,yes,yes,unknown,5,jun,26,24,-1,0,unknown,0,0.0018
3,4,34,technician,single,secondary,no,-476,yes,no,unknown,27,may,92,4,-1,0,unknown,0,0.0016
4,5,37,entrepreneur,married,primary,no,62,no,no,cellular,31,jul,404,2,-1,0,unknown,0,0.1050


SIGNATE提出用データ作成

In [18]:
sample=pd.read_csv("submit_sample.csv",header=None)

sample[1]=pred['Score']
sample.to_csv("submit_bank_{}.csv".format('pycaret'),index=None,header=None)
print(sample.head())

   0       1
0  1  0.5441
1  2  0.6507
2  3  0.0018
3  4  0.0016
4  5  0.1050
